## Task- 1:Fine-Tune a Model: Description: Fine-tune a pre-trained CNN model (e.g., DeepFace or any of your choice) for age detection on the UTK dataset. Description: GUI is not necessary. It will be evaluated on the basis of accuracy score.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import cv2

In [2]:
# Load UTK Dataset
def load_utk_data(dataset_path, img_size=(128, 128)):
    images, ages = [], []
    for file_name in os.listdir(dataset_path):
        if not file_name.endswith(".jpg"):
            continue
        try:
            age = int(file_name.split('_')[0])  # Extract age from file name
            img = cv2.imread(os.path.join(dataset_path, file_name))
            img = cv2.resize(img, img_size)
            images.append(img)
            ages.append(age)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
    return np.array(images), np.array(ages)

# Load data
dataset_path = r"C:\Users\sreel\OneDrive\Desktop\Null Class\Dataset\UTKFace"
img_size = (128, 128)
images, ages = load_utk_data(dataset_path, img_size)
images = images / 255.0  # Normalize pixel values

In [3]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(images, ages, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [4]:
# Model definition
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=img_size + (3,))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(1, activation="linear")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="mean_squared_error",
              metrics=["mae"])

# Train model
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    epochs=10, batch_size=32)

Epoch 1/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 255s 456ms/step - loss: 296.2116 - mae: 12.7785 - val_loss: 159.8317 - val_mae: 9.6820
Epoch 2/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 5138s 10s/step - loss: 154.2934 - mae: 9.3930 - val_loss: 149.3391 - val_mae: 9.1774
Epoch 3/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 263s 426ms/step - loss: 138.7029 - mae: 8.8295 - val_loss: 143.9261 - val_mae: 9.1315
Epoch 4/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 281s 461ms/step - loss: 129.1865 - mae: 8.5132 - val_loss: 142.6745 - val_mae: 9.0162
Epoch 5/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 218s 409ms/step - loss: 125.3267 - mae: 8.3231 - val_loss: 141.7583 - val_mae: 8.9159
Epoch 6/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 206s 386ms/step - loss: 120.6480 - mae: 8.1204 - val_loss: 135.0704 - val_mae: 8.6860
Epoch 7/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 206s 386ms/step - loss: 118.4737 - mae: 8.0643 - val_loss: 138.5122 - val_mae: 8.9493
Epoch 8/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 203s 380ms/step - loss: 112.5820 - mae: 7.8953 - val_loss: 139.8465 - val_mae:

In [6]:
# Unfreeze base model and fine-tune
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="mean_squared_error",
              metrics=["mae"])

history_finetune = model.fit(X_train, y_train, 
                             validation_data=(X_val, y_val),
                             epochs=10 , batch_size=32)

Epoch 1/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 1048s 2s/step - loss: 269.6165 - mae: 12.1158 - val_loss: 723.9613 - val_mae: 23.1711
Epoch 2/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 953s 2s/step - loss: 83.2214 - mae: 6.8379 - val_loss: 297.4552 - val_mae: 13.6790
Epoch 3/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 916s 2s/step - loss: 57.6287 - mae: 5.7575 - val_loss: 313.9691 - val_mae: 13.9834
Epoch 4/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 922s 2s/step - loss: 43.0715 - mae: 4.9893 - val_loss: 205.1143 - val_mae: 11.2740
Epoch 5/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 920s 2s/step - loss: 37.2283 - mae: 4.5972 - val_loss: 219.7259 - val_mae: 11.8068
Epoch 6/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 455s 851ms/step - loss: 32.4699 - mae: 4.2732 - val_loss: 102.8921 - val_mae: 7.4225
Epoch 7/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 384s 719ms/step - loss: 28.6785 - mae: 4.0295 - val_loss: 102.1994 - val_mae: 7.5328
Epoch 8/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 381s 713ms/step - loss: 25.7308 - mae: 3.8147 - val_loss: 84.6179 - val_mae: 6.6034
Epoch 9/

In [7]:
# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on Test Set: {mae:.2f}")

# Save the model
model.save("age_detection_model.h5")

149/149 ━━━━━━━━━━━━━━━━━━━━ 27s 166ms/step


Mean Absolute Error on Test Set: 6.52
